# ROTAS test data

Kerry Wood (kerry.wood@asterism.ai)

In [1]:
import pandas as pd
import ctypes
import load_utils
import helpers

In [2]:
# this will only work if you have the full astrostandards
load_utils.init_all()

EnvConstDll: 0
TimeFuncDll: 0
AstroFuncDll: 0
TleDll: 0
SpVecDll: 0
VcmDll: 0
ExtEphemDll: 0
Sgp4PropDll: 0
SpPropDll: 0
ElOpsDll: 0
SatStateDll: 0
SensorDll: 0
ObsDll: 0
ObsOpsDll: 0
LamodDll: 0
RotasDll: 0
BatchDCDll: 0
b''


0

In [3]:
obs = pd.read_json('./48340_obs.json')
tle = pd.read_json('./48340_tle.json')
# load the obs
obs = obs[ ~obs['idSensor'].isnull() ].copy()
obs = obs[ ~obs['id'].isnull() ].copy()
# this is a data holder / formatter
OBSHELPER = helpers.astrostd_named_fields( load_utils.ObsDll, prefix='XA_OBS_' )

EFG = (ctypes.c_double * 3)()
def toEFG( R ):
    LLH = load_utils.AstroFuncDll.LLHToEFGPos( 
                                              (ctypes.c_double * 3)( R['senlat'],R['senlon'],R['senalt']),
                                              EFG
                                              )
    return list(EFG)

obs['efg'] = obs.apply( toEFG, axis=1 ) 
obs['obTime'] = pd.to_datetime( obs['obTime'] )
obs['ds50_utc'] = obs['obTime'].apply( lambda X :helpers.datetime_to_ds50(X,load_utils.TimeFuncDll) )

In [4]:
def addOb( R, fakeSatNo=777 ):
    satno = R['satNo']
    senx, seny, senz = R['efg']
    # data holders
    OBSHELPER.clear()
    OBSHELPER['XA_OBS_SECCLASS'] = 1
    OBSHELPER['XA_OBS_SATNUM']   = satno
    OBSHELPER['XA_OBS_SITETAG']  = satno
    OBSHELPER['XA_OBS_SPADOCTAG']  = satno
    OBSHELPER['XA_OBS_SENNUM']   = fakeSatNo
    OBSHELPER['XA_OBS_DS50UTC']  = R['ds50_utc']
    OBSHELPER['XA_OBS_ELORDEC']  = R['declination']
    OBSHELPER['XA_OBS_AZORRA']   = R['ra']
    OBSHELPER['XA_OBS_POSX']     = senx
    OBSHELPER['XA_OBS_POSY']     = seny
    OBSHELPER['XA_OBS_POSZ']     = senz
    OBSHELPER['XA_OBS_OBSTYPE']  = 9
    OBSHELPER['XA_OBS_TRACKIND'] = 3 # start of track
    OBSHELPER['XA_OBS_YROFEQNX'] = 2 # J2K equinox
    R['asObId']                 = load_utils.ObsDll.ObsAddFrArray( OBSHELPER.getData() )
    return R['asObId']

# add the obs to the astrostandards
load_utils.ObsDll.ObsRemoveAll()
obs['asObId'] = obs.apply( addOb, axis=1 )

In [5]:
# init the sensor (fake sat number must be same as above)
load_utils.SensorDll.SensorSetLocAll( 777, 0, 0, (ctypes.c_double * 3)(0,0,0), load_utils.Cstr('TEST',24), 99999, 0 )

0

In [6]:
this_tle = tle.iloc[0]
load_utils.TleDll.TleRemoveAllSats()
as_tle   = load_utils.TleDll.TleAddSatFrLines( 
    load_utils.Cstr( this_tle['line1'], 512 ),
    load_utils.Cstr( this_tle['line2'], 512 ) )
assert 0 == load_utils.SatStateDll.SatStateInitSat( as_tle )

In [7]:
# data holders
satelt = (ctypes.c_double * 9)()
obelt  = (ctypes.c_double * 9)()
b3s    = load_utils.Cstr('',512)
# get a ROTAS holder
OBSRESHELPER = helpers.astrostd_named_fields( load_utils.RotasDll, prefix='XA_OBSRES_' )

# assume that TLE is loaded and the sensor is setup...
def ROTASob( R ):
    assert 0 == load_utils.RotasDll.RotasComputeObsResiduals( R['asObId'], as_tle, OBSRESHELPER.data, satelt, obelt )
    return OBSRESHELPER.toDict()

rv = obs.apply( ROTASob, axis=1 ).values
rv = pd.DataFrame.from_records(rv)
total = pd.concat( (obs, rv), axis=1 )

In [8]:
total

,id,classificationMarking,obTime,idOnOrbit,satNo,trackId,origSensorId,uct,ra,declination,...,XA_OBSRES_OBSRA,XA_OBSRES_OBSDEC,XA_OBSRES_LON,XA_OBSRES_POSU,XA_OBSRES_POSV,XA_OBSRES_POSW,XA_OBSRES_CHI,XA_OBSRES_ANGSEP,XA_OBSRES_TDOA,XA_OBSRES_FDOA
1,ef98169c-81a0-4451-b46c-f0148607427d,U//PR-PPEC-OBS,2025-04-30 04:24:03.127392+00:00,48340.0,48340.0,NaN,NaN,NaN,67.757179,55.858787,...,68.498746,55.759370,-0.116600,-0.109142,-13.281094,1.465494e-13,0.0,0.350950,0.0,0.0
2,2a31ad82-5100-4365-8bbb-d992e8bd4445,U//PR-PPEC-OBS,2025-04-30 04:24:02.053440+00:00,48340.0,48340.0,NaN,NaN,NaN,67.983358,55.704750,...,68.900169,55.480983,-0.116067,-0.091598,-13.264131,-1.060485e-12,0.0,0.350591,0.0,0.0
3,ced52d21-ec34-423a-84e5-94eeddbfc81a,U//PR-PPEC-OBS,2025-04-30 04:24:00.121536+00:00,48340.0,48340.0,NaN,NaN,NaN,68.386205,55.427278,...,69.120360,55.325092,-0.115841,-0.097970,-13.262552,1.122658e-12,0.0,0.350592,0.0,0.0
4,723f6d3e-a825-403d-ba87-144a7b9c160c,U//PR-PPEC-OBS,2025-04-30 04:23:59.047583+00:00,48340.0,48340.0,NaN,NaN,NaN,68.607196,55.271889,...,69.340604,55.167108,-0.116249,-0.107759,-13.333812,-3.708145e-13,0.0,0.352513,0.0,0.0
5,db0d2407-e542-484c-aafb-9c75cd52e05c,U//PR-PPEC-OBS,2025-04-30 04:23:57.973632+00:00,48340.0,48340.0,NaN,NaN,NaN,68.828253,55.114409,...,69.513108,55.042304,-0.115821,-0.107437,-13.304181,5.320189e-13,0.0,0.351754,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,cf40bc32-c5a3-4fcc-a83c-3eb9cbfa9583,U//PR-PPEC-OBS,2025-04-30 04:21:47.161440+00:00,48340.0,48340.0,NaN,NaN,NaN,85.224802,34.462971,...,85.721441,34.307893,-0.092131,-0.106111,-13.104981,-1.039169e-13,0.0,0.308216,0.0,0.0
128,5d24cd5e-2136-4793-a7f6-dde9ebe55f7a,U//PR-PPEC-OBS,2025-04-30 04:21:46.087488+00:00,48340.0,48340.0,NaN,NaN,NaN,85.301145,34.294344,...,85.796967,34.139175,-0.092102,-0.104142,-13.122559,-4.307665e-14,0.0,0.308041,0.0,0.0
129,203b62f2-4697-428d-9219-2c198654b7f7,U//PR-PPEC-OBS,2025-04-30 04:21:45.014399+00:00,48340.0,48340.0,NaN,NaN,NaN,85.377263,34.125811,...,85.856919,34.005224,-0.091834,-0.088062,-13.101655,3.197442e-14,0.0,0.307004,0.0,0.0
130,bc3369c0-bbeb-4aad-9a11-4d910b09b35d,U//PR-PPEC-OBS,2025-04-30 04:21:44.155584+00:00,48340.0,48340.0,NaN,NaN,NaN,85.437684,33.992008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
total.to_csv('./rotas_results.csv',index=False)